In [1]:
import pandas as pd
import datetime as datetime
import numpy as np
import copy
from RSU_functions import * 

In [2]:
# If you want to change your marginal income tax (above 72k of taxable income, it is at 41%). Not really needed
tax_info_dict = {'TMI_IR': 0.3, 'cotisation': 0.172, 'flat_tax_plus_value': 0.3, 'deductible_csg':0.0680}

# Put here all sales and vesting event from EquatePlus
please put all event until end of 2020, including sales / vesting from before (other years). If you have sales in 2019, either rely on the built in function or please du check yourself how you reported them before. 

If you sold everything you had at one point, before 2019 then you can put only vesting and transaction after this big sale event.

If you sold everything in 2019, please put all information in 2019 and even before if you want the CSG rebate computation, if you don't, you may start with all events starting after this "big sales"

In [3]:

vesting_event = [{'date':datetime.datetime(2019,12,13),'amount':225,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,3,13),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,6,15),'amount':29,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,9,14),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,12,14),'amount':28,'macron_law_id':0, 'event':'vesting'},
                ]
selling_event = [
    {'date':datetime.datetime(2020,8,3),'amount':282, 'event':'sale'},
    {'date':datetime.datetime(2020,12,16),'amount':56, 'event':'sale'}
]

Computation occurs here, please just run these cells

In [4]:
all_events_sorted = sorted(vesting_event + selling_event, key = lambda k: k['date'])

In [5]:
portfolio = {}
portfolio['available_stock'] = []
portfolio['sale_event_recap'] = []
for event in all_events_sorted:
    if event['event'] == 'vesting':
        portfolio['available_stock'].append(copy.deepcopy(event))
    if event['event'] == 'sale':
        available_stock_after_transaction, sale_event_recap = get_sales_result(event, portfolio,
                                                                               matching_method = get_sale_order_from_optionality)
        portfolio['available_stock'] = available_stock_after_transaction
        portfolio['sale_event_recap'].append(sale_event_recap)
        

# Results to put in your taxation returns
Ici, toute les infos sont printer pour remplir le formulaire No 2074. 
pour 511 mettre Action criteo et equateplus.
Vesting amount with moins value, rebate etc... are for debugging purposes, please look if they make sense.

In [6]:
tax = 0
Capital_gain_or_loss = 0
vesting_gain_before_rebate_and_moins_value = 0
rebate_amount = 0
total_selling_price = 0
csg_rebate = 0
Capital_gain_or_loss_2019 = 0
rebate_amount_2019 = 0
vesting_gain_before_rebate_and_moins_value_2019 = 0
for event in portfolio['sale_event_recap']:
    for ev in event:            
        if ev['date de la cession (513)'].year ==2020:
            print('\n ----------------NOUVELLE TRANSACTION ---------- \n')
            for key, value in ev.items():
                print(  key, value)
            tax +=ev['tax']
            Capital_gain_or_loss += ev['resultat']
            rebate_amount += ev['rebate_without_moins_value']
            vesting_gain_before_rebate_and_moins_value += ev['prix daquisition global (521 et 523)']
            total_selling_price += ev['montant global (516 et 518)']
            
        if ev['date de la cession (513)'].year ==2019:
            #here it is to compute CSG than can be dedudected from last year, box 6DE. Only what is 
            # subject to IR can be deducted, hence only the  vesting amount after rebate and moins value
            #will be deductible
            Capital_gain_or_loss_2019 += ev['resultat']
            rebate_amount_2019 += ev['rebate_without_moins_value']
            vesting_gain_before_rebate_and_moins_value_2019 += ev['prix daquisition global (521 et 523)']
        



 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2020-08-03 00:00:00
valeur unitaire de la cession (514) 11.572573767695719
nombre de titres cedes (515) 28
montant global (516 et 518) 324.03206549548014
prix ou valeur acquisition unitaire (520) 7.47478386167147
prix daquisition global (521 et 523) 209.29394812680115
resultat 114.73811736867897
vesting_amount_with_moins_value 209.29394812680115
rebate_without_moins_value 0.0
rebate 0
tax 133.20817872645384

 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2020-08-03 00:00:00
valeur unitaire de la cession (514) 11.572573767695719
nombre de titres cedes (515) 29
montant global (516 et 518) 335.60463926317584
prix ou valeur acquisition unitaire (520) 10.868212920998845
prix daquisition global (521 et 523) 315.1781747089665
resultat 20.42646455420934
vesting_amount_with_moins_value 315.1781747089665
rebate_without_moins_value 0.0
rebate 0
tax 154.89203782889498

 ----------------NOUVELL

Normally here you should have automatically computed the sum of "plus or moins value", and it should be equal to the value printed in the cell below. Using the overall plus ou moins value, then I will be able to remove it from the vesting price to reduce overall tax burden if negative. 
If there is a rebate, I uniformly apply the rebate to the vesting gain with the moins value.

In [7]:
print('The total capital gain or loss is {}'.format(Capital_gain_or_loss))

The total capital gain or loss is -582.5388523496347


In [8]:
if Capital_gain_or_loss >0:
    vesting_gain_after_rebate_and_moins_value = vesting_gain_before_rebate_and_moins_value - rebate_amount
    total_tax_to_pay_approx = vesting_gain_after_rebate_and_moins_value * (tax_info_dict['TMI_IR'] + tax_info_dict[
            'cotisation']) + Capital_gain_or_loss * tax_info_dict[
                         'flat_tax_plus_value']
    rebate = rebate_amount
else:
    rebate_percent = rebate_amount / vesting_gain_before_rebate_and_moins_value
    vesting_gain_after_rebate_and_moins_value = (vesting_gain_before_rebate_and_moins_value + Capital_gain_or_loss) * (1-rebate_percent)
    rebate = (vesting_gain_before_rebate_and_moins_value + Capital_gain_or_loss) * (rebate_percent)
    total_tax_to_pay_approx = vesting_gain_after_rebate_and_moins_value * (tax_info_dict['TMI_IR'] + tax_info_dict[
            'cotisation'])

In [9]:
if vesting_gain_after_rebate_and_moins_value > 300000:
    print('fill 1TT, form 2042 C with ', vesting_gain_after_rebate_and_moins_value - 300000 )
print('fill 1TZ, form 2042 C with ', vesting_gain_after_rebate_and_moins_value)
print('fill 1UZ , form 2042 C with the rebate in euro', rebate )
print('fill 3VG , form 2042 C with capital gain', max(0,Capital_gain_or_loss) )
print('fill 3VG , form 2042 C with capital loss', min(0,Capital_gain_or_loss) )


print('You sold for {} this amount of stocks'.format(np.round(total_selling_price,2)))
print('You will pay around  {} this amount of tax, sorry'.format(np.round(total_tax_to_pay_approx ,2)))

fill 1TZ, form 2042 C with  4124.898241574613
fill 1UZ , form 2042 C with the rebate in euro 0.0
fill 3VG , form 2042 C with capital gain 0
fill 3VG , form 2042 C with capital loss -582.5388523496347
You sold for 4124.9 this amount of stocks
You will pay around  1946.95 this amount of tax, sorry


# CSG deduction from previous year (2019)

If you ahve your previous tax filling, it should be equal to your 1TZ case of last year times the deductible CSG, which is at 6.8%. If you don't, you can use this, should work

In [10]:
if vesting_gain_before_rebate_and_moins_value_2019 ==0:
    print('sorry, no CSG to be deducted for this year!')
    vesting_gain_after_rebate_and_moins_value_2019 =0
else:
    if Capital_gain_or_loss_2019 >0:
        vesting_gain_after_rebate_and_moins_value_2019 = vesting_gain_before_rebate_and_moins_value_2019 - rebate_amount_2019
        deductible_csg = vesting_gain_after_rebate_and_moins_value_2019 * tax_info_dict['deductible_csg']
    else:
        rebate_percent_2019 = rebate_amount_2019 / vesting_gain_after_rebate_and_moins_value_2019
        vesting_gain_after_rebate_and_moins_value_2019 = vesting_gain_before_rebate_and_moins_value_2019 - rebate_amount_2019 * (1-rebate_percent_2019)
        deductible_csg = vesting_gain_after_rebate_and_moins_value_2019 * tax_info_dict['deductible_csg']

sorry, no CSG to be deducted for this year!


In [11]:
print('fill box 6DE', vesting_gain_after_rebate_and_moins_value_2019)

fill box 6DE 0
